# Produces dataset of project times from appended pipeline datasets ("all_quarters_merged.csv")

In [22]:
import pandas as pd
import logging
import dateutil
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 500)

In [23]:
logging.basicConfig(level=logging.INFO)

# Code Broken Down

In [26]:
df = pd.read_csv("cleaned/all_quarters_merged.csv")

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1,2,6,12,16,17,20,21,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,52,53,54,62,64,81,83,90,93,94,95,96,98,99,100,101,102,103,104,105,109,115,116,117,118,119,120,121,122,123,124,125,129,136,142,144,145,146,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
# keep only variables of interest
list = ['address', 'apn', 'best_date', 'best_stat','firstfiled', 'report_quarter', 'report_year', 'units', 'unitsnet', 'dbi_permit', 'x', 'y']
df = df[list]

In [28]:
#consolidate status categories. Start with 3 for now
def status_function(value):
    if value['best_stat']=="CONSTRUCTION":
        field = 'Under Construction'
    elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
        field = 'Building Permit Approved'
    else:
        field = 'Proposed'
    return field
        
df['status']=df.apply(status_function, axis=1)

In [29]:
gb = df.groupby(['apn', 'address'])

In [30]:
gb.groups

{('000APN',
  'TOURIST/RESIDENTIAL HOTEL CONVERSION PROJECT'): Int64Index([15852], dtype='int64'),
 ('3702053', '1169 MARKET ST'): Int64Index([186, 925, 1881], dtype='int64'),
 ('3776092',
  '246 RITCH ST'): Int64Index([290, 824, 1676, 2771, 3230, 3967, 4711, 5661, 6184, 7113], dtype='int64'),
 ('1805006',
  '1433 48TH AV'): Int64Index([6688, 7630, 8444, 9305, 10123, 11123, 12200, 13313, 14557, 15340,
             16198],
            dtype='int64'),
 ('5324010', '1466 PALOU AV'): Int64Index([16222], dtype='int64'),
 ('3640079',
  '3040 24TH ST'): Int64Index([7667, 8562, 9429, 10236, 11260, 11515, 13245, 14490, 15688, 16688], dtype='int64'),
 ('3723115',
  '770 FOLSOM ST'): Int64Index([5160, 5964, 6611, 7396, 8229, 9227, 10046, 11298, 12371, 13483,
             14667],
            dtype='int64'),
 ('1667024',
  '4820 FULTON ST'): Int64Index([11654, 13060, 14237, 15679, 16906], dtype='int64'),
 ('5825032', '782-786 ANDOVER ST'): Int64Index([12675], dtype='int64'),
 ('6795A029',
  '203 CO

In [67]:
group_df = gb.get_group(('3755079','10 HALLAM ST'))

In [68]:
group_df = group_df.sort('best_date')
group_df.reset_index(drop=True, inplace=True)

/Users/briangoggin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [69]:
group_df

,address,apn,best_date,best_stat,firstfiled,report_quarter,report_year,units,unitsnet,dbi_permit,x,y,status
0,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24,3,2011,1.0,NaN,2.00811E+11,-122.408590,37.775677,Proposed
1,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24,4,2011,1.0,NaN,200811247224,-122.408590,37.775677,Proposed
2,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24 08:00:00,1,2012,1.0,1.0,200811247224,37.775677,-122.408590,Proposed
3,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24 00:00:00,2,2012,1.0,1.0,200811247224,37.775677,-122.408590,Proposed
4,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24,3,2012,1.0,1.0,2.00811E+11,-122.408590,37.775677,Proposed
5,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24,4,2012,1.0,1.0,200811247224,37.775677,-122.408590,Proposed
6,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24,1,2013,1.0,1.0,200811247224,37.775677,-122.408590,Proposed
7,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24,2,2013,1.0,1.0,200811247224,37.775677,-122.408590,Proposed
8,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24,3,2013,1.0,1.0,200811247224,37.775677,-122.408590,Proposed
9,10 HALLAM ST,3755079,2008-11-24,BP FILED,2008-11-24,4,2013,1.0,1.0,200811247224,37.775677,-122.408590,Proposed


In [ ]:
# identify building permit ID. Then fill in rest of quarters with this permit ID.
building_permit=np.nan
for index, row in group_df.iterrows():
    if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
        building_permit=building_permit
    else:
        building_permit = row['dbi_permit']

In [ ]:
building_permit

In [ ]:
group_df['dbi_permit']=building_permit

In [ ]:
#Identify completion quarter
for index, row in group_df.iterrows():
    if row['best_stat'] !='CONSTRUCTION':
        comp_quarter= np.nan
        comp_year=np.nan
    elif row['best_stat'] == 'CONSTRUCTION':
        comp_quarter = row['report_quarter']
        comp_year = row['report_year']

if pd.isnull(comp_quarter):
    pass
if comp_quarter == 4 and comp_year == 2016:
    comp_quarter = np.nan
elif comp_quarter == 4:
    comp_quarter = 1
    comp_year = 1+comp_year
else:
    comp_quarter = 1+comp_quarter
    

In [ ]:
if comp_quarter ==1:
    comp_daymth= '01/01'
elif comp_quarter ==2:
    comp_daymth= '04/01'
elif comp_quarter==3:
    comp_daymth= '07/01'
elif comp_quarter == 4:
    comp_daymth= '10/01'
elif pd.isnull(comp_quarter):
    comp_daymth=np.nan
    
if pd.isnull(comp_daymth):
    comp_date = np.nan
else:
    comp_date = comp_daymth + "/" + str(comp_year)

In [ ]:
comp_date

In [ ]:
#Identify earliest "firstfiled" date
firstfiled = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['firstfiled']):
        continue
    else:
        if len(firstfiled) == 0:
            firstfiled = row['firstfiled']
        else:
            continue

In [ ]:
#Identify earliest best date
earliest_BD = ''
for index, row in group_df.iterrows():
    if pd.isnull(row['best_date']):
        continue
    else:
        if len(earliest_BD) == 0:
            earliest_BD = row['best_date']
        else:
            continue

In [ ]:
firstfiled

In [ ]:
earliest_BD

In [ ]:
# Finalize first date variable (minimum of earliest best_date and firstfiled)
if firstfiled =='' and earliest_BD !='':
    first_date = earliest_BD
elif earliest_BD=='' and firstfiled !='':
    first_date = firstfiled
elif firstfiled !='' and earliest_BD !='':
    first_date = min(firstfiled, earliest_BD)

In [ ]:
first_date

In [ ]:
# initiate variables. Groups without these dates are blank for these variables.
BP_date = ''
con_date = ''

#Identify first date for all status categories
m=0
for index, row in group_df.iterrows():
    if m == 0:
        status_previous = 'blah'
    if row['status']=='Building Permit Approved':
        if index == 0:
            BP_date = row['best_date']
        elif index !=0:
            if status_previous =='Building Permit Approved':
                BP_date=BP_date
            else:
                BP_date = row['best_date']
    elif row['status']=='Under Construction':
        if index == 0:
            con_date = row['best_date']
        elif index !=0:
            if status_previous =='Under Construction':
                con_date=con_date
            else:
                con_date = row['best_date']
    status_previous = row['status']
    m=m+1

In [ ]:
print (BP_date, con_date)

In [ ]:
last_row = group_df.tail(1).copy() #most recent row

In [ ]:
last_row

In [ ]:
last_row['latest_project_record_date'] = last_row.best_date
last_row['first_project_record_date'] = group_df.iloc[0].best_date
last_row['latest_project_status'] = last_row.best_stat

In [ ]:
last_row

In [ ]:
## Store a parseable list of all the project states and the dates those states were reported
last_row['project_dates'] = str(tuple(group_df.best_date))
last_row['project_statuses'] = str(tuple(group_df.best_stat))

In [ ]:
last_row

In [ ]:
## Store the project duration in days
latest = last_row.latest_project_record_date.iloc[0]
first = last_row.first_project_record_date.iloc[0]


In [ ]:
if not (pd.isnull(latest) or pd.isnull(first)):
    last_row['project_duration_days'] = (dateutil.parser.parse(latest) - dateutil.parser.parse(first)).days

In [ ]:
last_row

# Actual Code

In [72]:
def convert_to_one_record_per_project(df):
    """
    Group the dataset by (apn, address) and then emit one row per (apn, address) pair.
    best_date and best_stat will be converted into arrays.
    All other attributes will be taken from the record with the most recent `best_date` attribute.
    """
    gb = df.groupby(['apn', 'address'])
    for k in gb.groups:
        group_df = gb.get_group(k)
        group_df = group_df.sort('best_date')
        last_row = group_df.tail(1).copy()
        
        # identify building permit ID. Then fill in rest of quarters with this permit ID.
        building_permit=np.nan
        for index, row in group_df.iterrows():
            if pd.isnull(row['dbi_permit']) and not pd.isnull(building_permit):
                building_permit=building_permit
            else:
                building_permit = row['dbi_permit']
                
        #Identify completion quarter for those projects that have reached completion
        for index, row in group_df.iterrows():
            if row['best_stat'] !='CONSTRUCTION':
                comp_quarter= np.nan
                comp_year=np.nan
            elif row['best_stat'] == 'CONSTRUCTION':
                comp_quarter = row['report_quarter']
                comp_year = row['report_year']

        if pd.isnull(comp_quarter):
            pass
        if comp_quarter == 1 and comp_year == 2016:
            comp_quarter = np.nan
        elif comp_quarter == 4:
            comp_quarter = 1
            comp_year = 1+comp_year
        else:
            comp_quarter = 1+comp_quarter
            
        if comp_quarter ==1:
            comp_daymth= '01/01'
        elif comp_quarter ==2:
            comp_daymth= '04/01'
        elif comp_quarter==3:
            comp_daymth= '07/01'
        elif comp_quarter == 4:
            comp_daymth= '10/01'
        elif pd.isnull(comp_quarter):
            comp_daymth=np.nan

        if pd.isnull(comp_daymth):
            comp_date = np.nan
        else:
            comp_date = comp_daymth + "/" + str(comp_year)
        
        #Identify earliest "firstfiled" date
        firstfiled = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['firstfiled']):
                continue
            else:
                if len(firstfiled) == 0:
                    firstfiled = row['firstfiled']
                else:
                    continue
        
        #Identify earliest best date
        earliest_BD = ''
        for index, row in group_df.iterrows():
            if pd.isnull(row['best_date']):
                continue
            else:
                if len(earliest_BD) == 0:
                    earliest_BD = row['best_date']
                else:
                    continue
        
        # Finalize first date variable (minimum of earliest best_date and firstfiled)
        if firstfiled =='' and earliest_BD !='':
            first_date = earliest_BD
        elif earliest_BD=='' and firstfiled !='':
            first_date = firstfiled
        elif firstfiled !='' and earliest_BD !='':
            first_date = min(firstfiled, earliest_BD)
            
        # initiate variables. Groups without these dates are blank for these variables.
        BP_date = ''
        con_date = ''

        #Identify first date for all status categories
        m=0
        for index, row in group_df.iterrows():
            if m == 0:
                status_previous = 'blah'
            if row['status']=='Building Permit Approved':
                if index == 0:
                    BP_date = row['best_date']
                elif index !=0:
                    if status_previous =='Building Permit Approved':
                        BP_date=BP_date
                    else:
                        BP_date = row['best_date']
            elif row['status']=='Under Construction':
                if index == 0:
                    con_date = row['best_date']
                elif index !=0:
                    if status_previous =='Under Construction':
                        con_date=con_date
                    else:
                        con_date = row['best_date']
            status_previous = row['status']
            m=m+1
        
        last_row['dbi_permit']= building_permit
        last_row['comp_date']=comp_date
        last_row['BP_date'] = BP_date
        last_row['con_date'] = con_date
        last_row['first_date']=first_date
        last_row['latest_project_record_date'] = last_row.best_date
        last_row['first_project_record_date'] = group_df.iloc[0].best_date
        last_row['latest_project_status'] = last_row.best_stat

        ## Store a parseable list of all the project states and the dates those states were reported
        last_row['project_dates'] = str(tuple(group_df.best_date))
        last_row['project_statuses'] = str(tuple(group_df.best_stat))

        ## Store the project duration in days
        if not (pd.isnull(comp_date) or pd.isnull(first_date)):
            last_row['project_duration_days'] = (dateutil.parser.parse(comp_date) - dateutil.parser.parse(first_date)).days

        yield last_row


def main():
    df = pd.read_csv("cleaned/all_quarters_merged.csv")
    #consolidate status categories. Start with 3 for now
    def status_function(value):
        if value['best_stat']=="CONSTRUCTION":
            field = 'Under Construction'
        elif (value['best_stat']=='BP APPROVED') | (value['best_stat']=='BP ISSUED') | (value['best_stat']=='BP REINSTATED'):
            field = 'Building Permit Approved'
        else:
            field = 'Proposed'
        return field

    df['status']=df.apply(status_function, axis=1)
    
    #keep only variables we want
    list = ['address', 'apn', 'best_date', 'best_stat','status', 'firstfiled', 'report_quarter', 'report_year','units','unitsnet', 'dbi_permit', 'x', 'y']
    df = df[list]
    new_df = pd.concat(convert_to_one_record_per_project(df))
    logging.info("Writing output ({} rows, {} cols) to data/cleaned/all_quarters__one_record_per_project.csv".format(*new_df.shape))
    new_df.to_csv("cleaned/all_quarters__one_record_per_project.csv")
    return new_df

In [73]:
new_df = main()

/Users/briangoggin/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (1,2,6,12,16,17,20,21,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,53,54,55,63,65,73,75,77,78,79,80,81,82,84,91,94,95,96,97,99,100,101,102,103,104,105,106,110,116,117,118,119,120,121,122,123,124,125,126,130,137,143,144,145,146,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/Users/briangoggin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [6]:
#First, filter out those projects that are exclusively non-residential (defined as those without units)
new_df = new_df[new_df['units'] > 0]

In [7]:
#Next, keep only those that reached completion at some point over the time period
new_df=new_df[pd.notnull(new_df['comp_date'])]

# Explore Duplicate Data

In [8]:
#Finally, filter out records with duplicate BP Ids
new_df[new_df.duplicated('dbi_permit', keep=False)]['address'].unique()


array(['1420 MISSION ST', 'HP SHIPYARD PHASE 1 (UNDER CONSTRUCTION)',
       '233 - 237 SHIPLEY ST', '2101 & 2155 WEBSTER STREET',
       '1400 07TH ST', '690 MONTEREY BL',
       '5 SHIPLEY ST / 935 FOLSOM ST / 77 FALMOUTH ST', '4 OCTAVIA ST',
       'HUNTERS POINT SHIPYARD PHASE I', '650 INDIANA ST (BLDG 2 OF 2)',
       '233-237 SHIPLEY ST', '4132 03RD ST', '1000 16TH ST',
       '1078 HAMPSHIRE ST', '800 BROTHERHOOD', '1169 MARKET ST',
       '8 OCTAVIA ST', '447 - 453 LINDEN ST', '650 INDIANA ST',
       '55 LAGUNA STREET', 'PARCEL F', '1490 OCEAN AV', '55 LAGUNA ST',
       '1400 MISSION ST', '218 BUCHANAN ST', '57 TEHAMA ST',
       '55 LAGUNA ST (BLDG 2)', '344 - 388 FULTON ST', '690 MONTEREY BLVD',
       '1446 OCEAN AV', '800 BROTHERHOOD WAY', '1239 TURK ST',
       '1100 GOLDEN GATE AV', '2155 WEBSTER ST', '4132 THIRD STREET',
       '1190 MISSION ST', '41 TEHAMA ST', '447 LINDEN ST',
       '1076 HAMPSHIRE ST', '236 - 238 SHIPLEY ST / 77 FALMOUTH ST'], dtype=object)

In [50]:
#explore duplicated
new_df[new_df['address']=='55 LAGUNA STREET']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
12071,,55 LAGUNA STREET,0857001,2015-10-30,CONSTRUCTION,10/01/2015,2014-06-26,201209059006,2006-10-31 08:00:00,2008-01-17,2006-10-31,2015-10-30,CONSTRUCTION,"('2008-01-17', '2008-01-17', '2008-01-17', '20...",3256.0,"('PL APPROVED', 'PL APPROVED', 'PL APPROVED', ...",3,2015,Under Construction,450.0,450.0,37.77235,-122.426279


In [51]:
# Same project, different address-apn combination—verified by looking at each address in PIM
new_df[new_df['dbi_permit']=='201209059006']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
12071,,55 LAGUNA STREET,0857001,2015-10-30,CONSTRUCTION,10/01/2015,2014-06-26,201209059006,2006-10-31 08:00:00,2008-01-17,2006-10-31,2015-10-30,CONSTRUCTION,"('2008-01-17', '2008-01-17', '2008-01-17', '20...",3256.0,"('PL APPROVED', 'PL APPROVED', 'PL APPROVED', ...",3,2015,Under Construction,450.0,450.0,37.77235,-122.426279
9055,2013-06-11,218 BUCHANAN ST,0857001A,2015-03-19,CONSTRUCTION,01/01/2015,2013-08-07,201209059006,2012-09-05,2012-09-05,NaN,2015-03-19,CONSTRUCTION,"('2012-09-05', '2012-09-05', '2012-09-05', '20...",848.0,"('BP FILED', 'BP FILED', 'BP FILED', 'BP APPRO...",4,2014,Under Construction,191.0,191.0,37.77235,-122.426279


In [22]:
#explore duplicated
new_df[new_df['address']=='2155 WEBSTER ST']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
9106,,2155 WEBSTER ST,0629037,2015-03-18,CONSTRUCTION,01/01/2015,2015-03-18,201311222660,2013-11-22,2013-11-22,NaN,2015-03-18,CONSTRUCTION,"('2013-11-22', '2013-11-22', '2013-11-22', '20...",405.0,"('BP FILED', 'BP FILED', 'BP FILED', 'BP FILED...",4,2014,Under Construction,77.0,77.0,37.79057,-122.432337


In [41]:
new_df[new_df['dbi_permit']=='200705010136']

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
11114,,72 TOWNSEND ST,3789972,2015-07-21,CONSTRUCTION,07/01/2015,2015-07-21,200705010136,2015-07-21,2015-07-21,NaN,2015-07-21,CONSTRUCTION,"('2015-07-21',)",-20.0,"('CONSTRUCTION',)",2,2015,Under Construction,74.0,74.0,37.781388,-122.390353


In [36]:
new_df[new_df['project_duration_days']<0]

,BP_date,address,apn,best_date,best_stat,comp_date,con_date,dbi_permit,first_date,first_project_record_date,firstfiled,latest_project_record_date,latest_project_status,project_dates,project_duration_days,project_statuses,report_quarter,report_year,status,units,unitsnet,x,y
11114,,72 TOWNSEND ST,3789972,2015-07-21,CONSTRUCTION,07/01/2015,2015-07-21,200705010136,2015-07-21,2015-07-21,NaN,2015-07-21,CONSTRUCTION,"('2015-07-21',)",-20.0,"('CONSTRUCTION',)",2,2015,Under Construction,74.0,74.0,37.781388,-122.390353


# Manual Data Cleaning
Based off of identified duplicates in the previous section